In [13]:
import pyBeamSim
import numpy as np
import pandas as pd
import torch
import torch.nn as nn


class EncoderLstm(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLstm, self).__init__()
        self.hidden_size = hidden_size
        # self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)

    def forward(self, x, hidden):
        # embedded = self.embedding(x).view(1, 1, -1)
        output, hidden = self.lstm(x, hidden)
        return output, hidden

    def initHidden(self, batch_size):
        return (torch.zeros(1, batch_size, self.hidden_size),
                torch.zeros(1, batch_size, self.hidden_size))

# output_encoder shape: [batch_size, seq_len, hidden_size]
# hidden_encoder shape : [1, batch_size, hidden_size]

# output_encoder[:, -1, :] as input_decoder , hidden same as  , et seq_len = 1
# output_decoder  shape: [batch_size, seq_len, hidden_size]

class DecoderLstm(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DecoderLstm, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        # self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)

        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        # embedded = self.embedding(x).view(1, 1, -1)

        # x.shape~X[:, -1, :]: [batch_size, input_size]
        output, hidden = self.lstm(x.unsqueeze(1), hidden)
        # output_shape: [batch_size, seq_len=1, hidden_size]
        # hidden shape: [1, batch_size, hidden_size]
        output = self.fc(output.squeeze(1))
        # output_decoder  shape: [batch_size, output_size]
        return output, hidden

    def initHidden(self, batch_size):
        return (torch.zeros(1, batch_size, self.hidden_size),
                torch.zeros(1, batch_size, self.hidden_size))

# take input_encoder[:, -1, :] as input_decoder

class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, output_len):
        super(Model, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.output_len = output_len
        # without encoder initial
        self.encoder = EncoderLstm(input_size, hidden_size)
        self.decoder = DecoderLstm(input_size, hidden_size, output_size)

    def forward(self, x):
        # x.shape: [batch_size, seq_len, input_size] (1000, 4, 1)
        hidden_state = self.encoder.initHidden(x.shape[0])
        output_encoder, hidden_encoder = self.encoder(x, hidden_state)
        # (1000, 4, 30) ,(1, 1000, 30)

        hidden_decoder = hidden_encoder
        # more changes can do
        input_decoder = x[:, -1, :]   # (1000,1)

        total = []
        for i in range(self.output_len):
            output_decoder, hidden_decoder = self.decoder(input_decoder, hidden_decoder)
            # output_decoder.shape: [batch_size, output_size]
            total.append(output_decoder)
            input_decoder = output_decoder.unsqueeze(1)

        return torch.concat(total, dim=1)

In [14]:
encoder = EncoderLstm(1, 30)

In [15]:
decoder = DecoderLstm(1, 30, 1)

In [16]:
x = torch.rand(size=(1000, 4, 1))
x.shape

torch.Size([1000, 4, 1])

In [17]:
hidden_state = encoder.initHidden(x.shape[0])
hidden_state[0].shape

torch.Size([1, 1000, 30])

In [18]:
output_encoder, hidden_encoder = encoder(x, hidden_state)
hidden_encoder[0].shape

torch.Size([1, 1000, 30])

In [19]:
hidden_decoder = hidden_encoder
input_decoder = x[:, -1, :] 
input_decoder.shape

torch.Size([1000, 1])

In [23]:
total = []

In [29]:
output_decoder, hidden_decoder = decoder(input_decoder, hidden_decoder)
total.append(output_decoder)
output_decoder.shape

torch.Size([1000, 1])

In [30]:
len(total)

3